# Proyecto Final IH: Recomendador de Codewars|Github

In [ ]:
# https://www.codewars.com/users/leaderboard

In [3]:
# imports 

import pandas as pd
import numpy as np
import requests
import json
from bs4 import BeautifulSoup
from IPython.display import display

pd.options.display.max_columns = None

## Fase 1: Obtención de usuarios de forma iterativa

### Leaderboard
Top 500 de codewars

In [ ]:
# URL

url = 'https://www.codewars.com/users/leaderboard'
html = requests.get(url).content
soup = BeautifulSoup(html, "lxml")

In [ ]:
# usuarios de leaderboard
def get_top_500(url='https://www.codewars.com/users/leaderboard'): 
    html = requests.get(url).content
    soup = BeautifulSoup(html, "lxml")
    return {e.text for e in soup.select('tr a')}

# users = '\n'.join(get_top_500())
# set_top500
# users
# get_top_500()

In [ ]:
# Salvamos los nombres en un documento
def save_set_users(users, filename='../output/codewars-users.txt'): 
    ''' Save set/list of users to file '''
    with open(filename, 'w') as f: 
        f.write('\n'.join(users)) 
# save_set_users({'hola', 'adios'})

In [ ]:
def save_string_users(users, filename='../output/codewars-users.txt'): 
    ''' Save string of users to file '''
    with open(filename, 'w') as f: 
        f.write(users) 
# save_set_users(users)

In [ ]:
def add_set_users(users, filename='../output/codewars-users.txt'): 
    ''' Append to file '''
    with open(filename, 'a') as f: 
        f.write(users) 
# add_set_users(users)

In [ ]:
def load_set_users(filename='../output/codewars-users.txt'): 
    ''' Read from file '''
    with open(filename, 'r') as f: 
        users = f.readlines()
    return {user.strip() for user in users}
# len(load_set_users())

### Get Social

In [ ]:
# get users in clan

# https://www.codewars.com/users/albertogcmr/following
# https://www.codewars.com/users/albertogcmr/followers
# https://www.codewars.com/users/albertogcmr/allies

def get_social(user): 
    res = set()
    for link in ['following', 'followers', 'allies']: 
        url = 'https://www.codewars.com/users/{}/{}'.format(user, link)
        html = requests.get(url).content

        soup = BeautifulSoup(html, "lxml")
        res = res.union({e.text for e in soup.select('table a')})
    return res

# get_social('albertogcmr')

In [ ]:
### Get stats from user
'''
def get_all_stats(user): 
    url = 'https://www.codewars.com/users/{}'.format(user)
    html = requests.get(url).content
    soup = BeautifulSoup(html, "lxml")

    x = {}
    x['username'] = user
    for s in soup.select('.stat-box div'): 
        if s.text.split(':')[0] != 'Profiles': 
            x[s.text.split(':')[0].lower()] = s.text.split(':')[1]
        else: 
            try: 
                for e in s.find_all('a', href=True): 
                    if 'github' in e['href']: 
                        x['github'] = e['href']
                    if 'linkedin' in e['href']: 
                        x['linkedin'] = e['href']
            except: 
                x['github'] = ''
                x['linkedin'] = ''
    return x

get_all_stats('albertogcmr')

'''
pass

### Iteración para ampliar número de usuarios

A partir de la semilla inicial de 500 obtenemos casi 4000 usuarios de codewars

In [ ]:
top500 = get_top_500()
total = set().union(top500)
users = total

In [ ]:
print(len(total))

for i, user in enumerate(users): 
    total = total.union(get_social(user))
    print(i, '\t', len(total))

print(len(total))

In [ ]:
# Me añado a mí mismo

total = total.union({'albertogcmr'})
for user in get_social('albertogcmr'): 
    total = total.union(get_social(user))
len(total)

### Persistencia de datos en un txt

In [ ]:
# save_set_users(users=total, filename='../output/no-borrar-usuarioscodewars-2.txt')
total = load_set_users('../output/no-borrar-usuarioscodewars-2.txt')

In [ ]:
len(total)

## Fase 2: Obtención de datos de usuario
### 2.1 API codewars

In [4]:
# GET user

def get_user_api(user='albertogcmr'): 
    ''' Obtenemos el json de la API de codewars '''
    url = 'https://www.codewars.com/api/v1/users/{}'.format(user)
    response = requests.get(url)
    return response.json()

get_user_api('albertogcmr')

{'username': 'albertogcmr',
 'name': 'Alberto García cobo',
 'honor': 727,
 'clan': 'Ironhack',
 'leaderboardPosition': 12892,
 'skills': ['java',
  'c',
  'python',
  'jquery',
  'javascript',
  'crm',
  'erp',
  'itil'],
 'ranks': {'overall': {'rank': -4,
   'name': '4 kyu',
   'color': 'blue',
   'score': 888},
  'languages': {'python': {'rank': -4,
    'name': '4 kyu',
    'color': 'blue',
    'score': 857},
   'sql': {'rank': -7, 'name': '7 kyu', 'color': 'white', 'score': 24},
   'c': {'rank': -8, 'name': '8 kyu', 'color': 'white', 'score': 10}}},
 'codeChallenges': {'totalAuthored': 0, 'totalCompleted': 89}}

In [5]:
def get_value_from_json(user_json, value='username'): 
    ''' 
    value en nivel 1 del json: 
    possible values = ['username', 'honor', 'clan', 'leaderboardPosition', 'skills']
    '''
    return user_json[value]

def get_score_language(user_json, language): 
    try: 
        score = user_json['ranks']['languages'][language]['score']
    except: 
        score = 0
    return score


def get_scores(user_json): 
    res = {}
    for lang in get_languages(): 
        res[lang] = get_score_language(user_json, lang)
    return res

# get_score_language(user_json=alberto, language='sca')
# get_value_from_json(alberto, 'skills')
# get_scores(alberto)

### 2.2 Web Scrapping codewars

Get stats from user

In [6]:
def get_all_stats(user): 
    url = 'https://www.codewars.com/users/{}'.format(user)
    html = requests.get(url).content
    soup = BeautifulSoup(html, "lxml")

    x = {}
    x['username'] = user
    for s in soup.select('.stat-box div'): 
        if s.text.split(':')[0] != 'Profiles': 
            x[s.text.split(':')[0].lower()] = s.text.split(':')[1]
        else: 
            try: 
                for e in s.find_all('a', href=True): 
                    if 'github' in e['href']: 
                        x['github'] = e['href']
                    if 'linkedin' in e['href']: 
                        x['linkedin'] = e['href']
            except: 
                x['github'] = ''
                x['linkedin'] = ''
    return x

get_all_stats('albertogcmr')

{'username': 'albertogcmr',
 'name': 'Alberto García cobo',
 'clan': 'Ironhack',
 'skills': 'java, c, python, jquery, javascript, crm, erp, itil',
 'member since': 'Oct 2018',
 'last seen': 'Dec 2018',
 'github': 'https://github.com/albertogcmr',
 'linkedin': 'https://www.linkedin.com/in/albertogarciacobo/',
 'following': '593',
 'followers': '591',
 'allies': '591',
 'rank': '4 kyu',
 'honor': '727',
 'leaderboard position': '#12,892',
 'honor percentile': '98th',
 'total completed kata': '89',
 'total languages trained': '3',
 'highest trained': 'Python (4 kyu)',
 'most recent': 'Python',
 'comments': '0 (0 replies)',
 'collections': '0',
 'kumite': '0',
 'translations': '0 (0 approved)'}

In [7]:
def get_languages(url='https://www.codewars.com/kata/latest/my-languages', ignore={'all', 'my languages'}): 
    html = requests.get(url).content
    soup = BeautifulSoup(html, "lxml")
    return {e.text.lower() for e in soup.select('#language_filter option')}.difference(ignore) 

get_languages()

{'bf (beta)',
 'c',
 'c#',
 'c++',
 'clojure',
 'coffeescript',
 'crystal',
 'dart',
 'elixir',
 'elm (beta)',
 'erlang (beta)',
 'f#',
 'fortran (beta)',
 'go',
 'groovy (beta)',
 'haskell',
 'java',
 'javascript',
 'julia (beta)',
 'kotlin (beta)',
 'lua (beta)',
 'nasm (beta)',
 'nim (beta)',
 'objective-c (beta)',
 'ocaml (beta)',
 'php',
 'powershell (beta)',
 'purescript (beta)',
 'python',
 'r (beta)',
 'ruby',
 'rust',
 'scala (beta)',
 'shell',
 'solidity (beta)',
 'sql',
 'swift',
 'typescript'}

In [ ]:
def get_row(user): 
    ''' 
    Crea un diccionario con los datos recopilados que se puede
    añadir como fila en nuestro dataframe de pandas
    '''
    user_json = get_user_api(user)
    res = get_all_stats(user)
    res.update(get_scores(user_json))
    return res

# ivan = get_row('ijcernicharo')
# alberto = get_row('albertogcmr')

In [ ]:
def create_row(data, new_user): 
    row = get_row(new_user)
    return data.append(other=row, ignore_index=True)

### 2.3 Creación de DataFrame

In [ ]:
df_users = pd.DataFrame()

In [ ]:
# Por cada uno de los usuarios recopilados, creamos un registro en df_users
total_500 = [u for u in total][0:1000]
for i, user in enumerate(total_500): 
    try: 
        df_users = create_row(data=df_users, new_user=user)
        print(i)
    except: 
        print('Fallo:', user)

df_users.head()

In [ ]:
df_users = create_row(data=df_users, new_user='ijcernicharo')
df_users = create_row(data=df_users, new_user='pablobarrio')
df_users.head()

In [ ]:
df_users.head(100)

In [ ]:
'linkedin' in df_users.columns

In [ ]:
df_users = create_row(data=df_users, new_user='albertogcmr')
df_users.head(7)

In [ ]:
'linkedin' in df_users.columns

In [ ]:
df_users.to_csv('../output/dataframe.csv')
df = pd.read_csv('../output/dataframe.csv')
df.head()

In [ ]:
df.games.value_counts()

### 2.4 Limpieza de datos

Tenemos más de 100 columnas de las que sólo queremos las concernientes a sus datos de programación

In [ ]:
# Nos vamos a quedar con las siguientes columnas
get_languages()
allies, clan, comments, followers, following, github, honor, last seen, 
leaderboard position # eliminar el #
member since, total completed kata, total languages trained, translations, username, avg. satisfaction rating, 
contributed kata, created, data structures, data types, fundamentals, graphs, kata approvals, 
total collected, total completions, total stars, skills, linkedin


## Fase 3: Persistencia de datos

In [ ]:
Guarda